In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 https://cloud.r-project.org/b

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 

In [6]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
schema=StructType().add("id_utilisateur","string").add("pays","string").add("profession", "string").add("genre",'string').add("age", "integer")
df=spark.createDataFrame([("A123",'Maroc',"Médecin","Femme", 33),("B234",'Canada',"Entrepreneur","Femme",35),("C345",'France',"Professeur", "Homme",40)],schema=schema)

In [7]:
df.show()

+--------------+------+------------+-----+---+
|id_utilisateur|  pays|  profession|genre|age|
+--------------+------+------------+-----+---+
|          A123| Maroc|     Médecin|Femme| 33|
|          B234|Canada|Entrepreneur|Femme| 35|
|          C345|France|  Professeur|Homme| 40|
+--------------+------+------------+-----+---+



In [9]:
def cat_age(age):
  if age < 40:
    return "Catégorie 1"
  else:
    return "Catégorie 2"

In [10]:
from pyspark.sql.functions import udf
age_udf=udf(cat_age,StringType())

In [12]:
df=df.withColumn('cat_age',age_udf(df['age']))
df.show()

+--------------+------+------------+-----+---+-----------+
|id_utilisateur|  pays|  profession|genre|age|    cat_age|
+--------------+------+------------+-----+---+-----------+
|          A123| Maroc|     Médecin|Femme| 33|Catégorie 1|
|          B234|Canada|Entrepreneur|Femme| 35|Catégorie 1|
|          C345|France|  Professeur|Homme| 40|Catégorie 2|
+--------------+------+------------+-----+---+-----------+



In [13]:
df.groupby("cat_age").count().show()

+-----------+-----+
|    cat_age|count|
+-----------+-----+
|Catégorie 1|    2|
|Catégorie 2|    1|
+-----------+-----+

